# Context Formatting

When an AI agent retrieves information from various sources - RAG systems, tool calls, memory, API responses - that information often arrives as raw, unstructured text. Before passing this content to the language model, we must decide how to format it. The format we choose is not cosmetic. It directly affects how well the model reads, reasons over, and uses the information.

This notebook covers five practical formatting options: **Markdown**, **JSON**, **XML**, **TOON** and **plain text**. Each has a different structure, a different level of verbosity, and works better for different types of content. We will rewrite the same retrieved context in all five formats, send it to the model, and compare the results.

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
import json
import os

### Initialize the language model

In [2]:
# Using gpt-4o-mini for cost-effective experimentation
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY", "").strip(), temperature=0)

## The problem: unformatted context
Retrieval systems often return chunks of raw text. When multiple chunks from different sources are concatenated and passed to the model without any structure, the model must simultaneously parse the content, figure out what belongs together, and reason about the task. This extra cognitive load leads to worse answers — the model misses relationships, conflates sources or gives incomplete responses.

Below is the same piece of information that a customer service agent might retrieve about a user, their order, and the return policy. First, we pass it raw and unformatted.

In [3]:
# The raw, unformatted context that retrieval returns
raw_context = """
customer name sarah chen gold member since 2022 email sarah@example.com 
order id 12345 product wireless headphones model XZ-200 purchase date 2023-08-15 
price 299 usd status shipped carrier ups tracking 1Z999AA10123456784 
return policy 30 days from delivery date original packaging required 
refund processed within 5-7 business days to original payment method 
warranty 2 years covers manufacturing defects does not cover physical damage
"""

TASK = "Can I return my order and how long will the refund take?"

prompt = f"{raw_context}\n\nQuestion: {TASK}"

response = llm.invoke([HumanMessage(content=prompt)])

print("RAW UNFORMATTED CONTEXT")
print("=" * 70)
print(raw_context)
print("\nMODEL RESPONSE:")
print("-" * 70)
print(response.content)

RAW UNFORMATTED CONTEXT

customer name sarah chen gold member since 2022 email sarah@example.com 
order id 12345 product wireless headphones model XZ-200 purchase date 2023-08-15 
price 299 usd status shipped carrier ups tracking 1Z999AA10123456784 
return policy 30 days from delivery date original packaging required 
refund processed within 5-7 business days to original payment method 
warranty 2 years covers manufacturing defects does not cover physical damage


MODEL RESPONSE:
----------------------------------------------------------------------
Yes, you can return your order within 30 days from the delivery date, provided that you have the original packaging. Since your order was shipped on August 15, 2023, you have until September 14, 2023, to initiate a return.

Once the return is processed, the refund will take 5-7 business days to be credited back to your original payment method.


The model can technically answer, but it has to work hard to separate the return policy facts from the customer facts and the order facts. With longer, noisier input, this degrades further. Formatting solves the problem before it reaches the model.

## Format 1: Markdown
Markdown uses `##` headers, `-` bullet points, and `**bold**` to impose a readable hierarchy on text. It is the closest to how humans naturally organize notes. Language models are trained on enormous amounts of Markdown (GitHub, documentation, Stack Overflow), so they parse it fluently.

**When it works well:**
- Mixed content with several logical sections (user info, order info, policy).
- Content that will be read by both humans and the model (debugging, logging).
- When we want the model to produce structured prose in response.

**When to avoid it:**
- When downstream code needs to parse the model's output programmatically (use JSON instead).
- Very short, simple context where headers add noise without benefit.

In [4]:
def format_as_markdown(customer: dict, order: dict, policy: dict) -> str:
    """
    Format retrieved context as Markdown.
    Uses headers for sections and bullet points for individual facts.

    Args:
        customer: Customer profile data
        order: Order details
        policy: Return and warranty policy

    Returns:
        Markdown-formatted context string
    """
    return f"""## Customer
- **Name:** {customer['name']}
- **Tier:** {customer['tier']}
- **Email:** {customer['email']}

## Order
- **Order ID:** {order['id']}
- **Product:** {order['product']}
- **Date:** {order['date']}
- **Price:** {order['price']}
- **Status:** {order['status']} via {order['carrier']}

## Return Policy
- **Window:** {policy['return_window']}
- **Condition:** {policy['condition']}
- **Refund timeline:** {policy['refund_timeline']}

## Warranty
- **Duration:** {policy['warranty_duration']}
- **Covers:** {policy['warranty_covers']}
- **Does not cover:** {policy['warranty_excludes']}"""


# Source data — same facts as the raw context above
customer_data = {
    "name": "Sarah Chen",
    "tier": "Gold member since 2022",
    "email": "sarah@example.com"
}

order_data = {
    "id": "#12345",
    "product": "Wireless Headphones XZ-200",
    "date": "2023-08-15",
    "price": "$299 USD",
    "status": "Shipped",
    "carrier": "UPS (tracking: 1Z999AA10123456784)"
}

policy_data = {
    "return_window": "30 days from delivery date",
    "condition": "Original packaging required",
    "refund_timeline": "5-7 business days to original payment method",
    "warranty_duration": "2 years",
    "warranty_covers": "Manufacturing defects",
    "warranty_excludes": "Physical damage"
}

markdown_context = format_as_markdown(customer_data, order_data, policy_data)
prompt = f"{markdown_context}\n\nQuestion: {TASK}"

response = llm.invoke([HumanMessage(content=prompt)])

print("MARKDOWN FORMATTED CONTEXT")
print("=" * 70)
print(markdown_context)
print("\nMODEL RESPONSE:")
print("-" * 70)
print(response.content)

MARKDOWN FORMATTED CONTEXT
## Customer
- **Name:** Sarah Chen
- **Tier:** Gold member since 2022
- **Email:** sarah@example.com

## Order
- **Order ID:** #12345
- **Product:** Wireless Headphones XZ-200
- **Date:** 2023-08-15
- **Price:** $299 USD
- **Status:** Shipped via UPS (tracking: 1Z999AA10123456784)

## Return Policy
- **Window:** 30 days from delivery date
- **Condition:** Original packaging required
- **Refund timeline:** 5-7 business days to original payment method

## Warranty
- **Duration:** 2 years
- **Covers:** Manufacturing defects
- **Does not cover:** Physical damage

MODEL RESPONSE:
----------------------------------------------------------------------
Yes, you can return your order since it falls within the 30-day return window from the delivery date. Your order was shipped on August 15, 2023, so you have until September 14, 2023, to initiate the return. Please ensure that the product is in its original packaging.

Once the return is processed, the refund will take 

The model now gives a focused, structured answer. The headers made it immediately clear which facts belong to return policy, which to the order, and which to the warranty - without any ambiguity.

## Format 2: JSON
JSON uses nested key-value pairs and arrays to represent structured data. It is the format of APIs and databases, so it is ideal when our retrieved context is already structured data — tool outputs, database rows, API responses.

**When it works well:**
- Tool outputs or API responses that are already structured.
- When we need the model to extract or compare specific fields precisely.
- When the model needs to produce structured output itself (JSON in, JSON out).

**When to avoid it:**
- For narrative or conversational content (JSON adds visual noise without benefit).
- When token efficiency matters - JSON keys repeat field names, increasing token count.

In [5]:
def format_as_json(customer: dict, order: dict, policy: dict) -> str:
    """
    Format retrieved context as JSON.
    Groups data into a single nested structure with clear keys.

    Args:
        customer: Customer profile data
        order: Order details
        policy: Return and warranty policy

    Returns:
        JSON-formatted context string
    """
    context = {
        "customer": {
            "name": customer["name"],
            "tier": customer["tier"],
            "email": customer["email"]
        },
        "order": {
            "id": order["id"],
            "product": order["product"],
            "purchase_date": order["date"],
            "price": order["price"],
            "status": order["status"],
            "carrier": order["carrier"]
        },
        "policies": {
            "return": {
                "window": policy["return_window"],
                "condition": policy["condition"],
                "refund_timeline": policy["refund_timeline"]
            },
            "warranty": {
                "duration": policy["warranty_duration"],
                "covers": policy["warranty_covers"],
                "excludes": policy["warranty_excludes"]
            }
        }
    }
    # indent=2 makes it readable for both the model and developers
    return json.dumps(context, indent=2)


json_context = format_as_json(customer_data, order_data, policy_data)
prompt = f"{json_context}\n\nQuestion: {TASK}"

response = llm.invoke([HumanMessage(content=prompt)])

print("JSON FORMATTED CONTEXT")
print("=" * 70)
print(json_context)
print("\nMODEL RESPONSE:")
print("-" * 70)
print(response.content)

JSON FORMATTED CONTEXT
{
  "customer": {
    "name": "Sarah Chen",
    "tier": "Gold member since 2022",
    "email": "sarah@example.com"
  },
  "order": {
    "id": "#12345",
    "product": "Wireless Headphones XZ-200",
    "purchase_date": "2023-08-15",
    "price": "$299 USD",
    "status": "Shipped",
    "carrier": "UPS (tracking: 1Z999AA10123456784)"
  },
  "policies": {
    "return": {
      "window": "30 days from delivery date",
      "condition": "Original packaging required",
      "refund_timeline": "5-7 business days to original payment method"
    },
    "warranty": {
      "duration": "2 years",
      "covers": "Manufacturing defects",
      "excludes": "Physical damage"
    }
  }
}

MODEL RESPONSE:
----------------------------------------------------------------------
Yes, you can return your order within 30 days from the delivery date, provided that you have the original packaging. Once the return is processed, the refund will take 5-7 business days to be credited back 

JSON is precise. The model can pinpoint exactly which field answers the question. Notice that the response references specific field values directly. This makes JSON the best choice when precision and field-level extraction matter.

## Format 3: XML
XML uses opening and closing tags (`<tag>...</tag>`) to wrap content. It is more verbose than Markdown or JSON, but each piece of information is unambiguously bounded. This makes it especially powerful when we need to mix structured data with longer prose sections - the tags tell the model exactly where one chunk of information ends and another begins.

Anthropic's own prompt engineering guides recommend XML tags for separating context sections, because the explicit open/close tags make boundaries impossible to miss.

**When it works well:**
- When context sections contain long prose that would look awkward in JSON.
- When we need absolutely unambiguous boundaries between sections.
- When mixing structured fields with free-text descriptions.

**When to avoid it:**
- When token count is a hard constraint (XML is the most verbose of the four).
- For purely numeric or tabular data (JSON is cleaner).

In [6]:
def format_as_xml(customer: dict, order: dict, policy: dict) -> str:
    """
    Format retrieved context as XML.
    Uses descriptive tag names to wrap each section and field.

    Args:
        customer: Customer profile data
        order: Order details
        policy: Return and warranty policy

    Returns:
        XML-formatted context string
    """
    return f"""<context>

  <customer>
    <name>{customer['name']}</name>
    <tier>{customer['tier']}</tier>
    <email>{customer['email']}</email>
  </customer>

  <order>
    <id>{order['id']}</id>
    <product>{order['product']}</product>
    <purchase_date>{order['date']}</purchase_date>
    <price>{order['price']}</price>
    <status>{order['status']}</status>
    <carrier>{order['carrier']}</carrier>
  </order>

  <return_policy>
    <window>{policy['return_window']}</window>
    <condition>{policy['condition']}</condition>
    <refund_timeline>{policy['refund_timeline']}</refund_timeline>
  </return_policy>

  <warranty>
    <duration>{policy['warranty_duration']}</duration>
    <covers>{policy['warranty_covers']}</covers>
    <excludes>{policy['warranty_excludes']}</excludes>
  </warranty>

</context>"""


xml_context = format_as_xml(customer_data, order_data, policy_data)
prompt = f"{xml_context}\n\nQuestion: {TASK}"

response = llm.invoke([HumanMessage(content=prompt)])

print("XML FORMATTED CONTEXT")
print("=" * 70)
print(xml_context)
print("\nMODEL RESPONSE:")
print("-" * 70)
print(response.content)

XML FORMATTED CONTEXT
<context>

  <customer>
    <name>Sarah Chen</name>
    <tier>Gold member since 2022</tier>
    <email>sarah@example.com</email>
  </customer>

  <order>
    <id>#12345</id>
    <product>Wireless Headphones XZ-200</product>
    <purchase_date>2023-08-15</purchase_date>
    <price>$299 USD</price>
    <status>Shipped</status>
    <carrier>UPS (tracking: 1Z999AA10123456784)</carrier>
  </order>

  <return_policy>
    <window>30 days from delivery date</window>
    <condition>Original packaging required</condition>
    <refund_timeline>5-7 business days to original payment method</refund_timeline>
  </return_policy>

  <warranty>
    <duration>2 years</duration>
    <covers>Manufacturing defects</covers>
    <excludes>Physical damage</excludes>
  </warranty>

</context>

MODEL RESPONSE:
----------------------------------------------------------------------
Yes, you can return your order within 30 days from the delivery date, provided that it is in its original packag

XML gives the model explicit boundaries via tag names. Even if the content inside a tag is a long paragraph, the model knows exactly where it starts and ends. This is particularly useful when some fields contain free-form text (e.g., a product description or a support note) that would break JSON formatting.

## Format 4: TOON (Token Oriented Object Notation)
TOON is a compact, token-efficient notation designed specifically for LLM context. Unlike the other formats, it is not optimised for human readability or interoperability, but for minimising token cost while preserving structure. It uses:
- `[section]` brackets to group related fields on one line.
- `key=value` pairs with no quotes or indentation.
- `|` pipe separators between fields on the same line.
- Short but readable key names.

This means the same information that takes 40 tokens in JSON takes roughly 15 tokens in TOON - a significant saving when context windows are under pressure or when we are paying per token.

**When it works well:**
- Large amounts of structured data where token budget is the primary constraint.
- Repeated context patterns (e.g., many similar records) where JSON/XML overhead multiplies.
- Internal agent pipelines where humans do not need to read the raw context.

**When to avoid it:**
- When the content includes long prose values that do not compress well into key=value.
- When a downstream system expects a standard format like JSON or XML.

In [7]:
def format_as_toon(customer: dict, order: dict, policy: dict) -> str:
    """
    Format retrieved context as TOON (Token Oriented Object Notation).
    Packs each section onto a single line using key=value pairs separated by pipes.
    No quotes, no indentation, no closing tags — purely token-efficient.

    Args:
        customer: Customer profile data
        order: Order details
        policy: Return and warranty policy

    Returns:
        TOON-formatted context string
    """
    # Each section is a bracketed label followed by pipe-separated key=value pairs.
    # Key names are kept short but still readable (e.g., 'ret' for return window).
    customer_line = (
        f"[cust] name={customer['name']}|"
        f"tier={customer['tier']}|"
        f"email={customer['email']}"
    )

    order_line = (
        f"[ord] id={order['id']}|"
        f"prod={order['product']}|"
        f"date={order['date']}|"
        f"price={order['price']}|"
        f"status={order['status']}|"
        f"carrier={order['carrier']}"
    )

    policy_line = (
        f"[policy] ret={policy['return_window']}|"
        f"cond={policy['condition']}|"
        f"refund={policy['refund_timeline']}|"
        f"warr={policy['warranty_duration']}|"
        f"covers={policy['warranty_covers']}|"
        f"excl={policy['warranty_excludes']}"
    )

    return "\n".join([customer_line, order_line, policy_line])


toon_context = format_as_toon(customer_data, order_data, policy_data)
prompt = f"{toon_context}\n\nQuestion: {TASK}"

response = llm.invoke([HumanMessage(content=prompt)])

print("TOON FORMATTED CONTEXT")
print("=" * 70)
print(toon_context)
print("\nMODEL RESPONSE:")
print("-" * 70)
print(response.content)

TOON FORMATTED CONTEXT
[cust] name=Sarah Chen|tier=Gold member since 2022|email=sarah@example.com
[ord] id=#12345|prod=Wireless Headphones XZ-200|date=2023-08-15|price=$299 USD|status=Shipped|carrier=UPS (tracking: 1Z999AA10123456784)
[policy] ret=30 days from delivery date|cond=Original packaging required|refund=5-7 business days to original payment method|warr=2 years|covers=Manufacturing defects|excl=Physical damage

MODEL RESPONSE:
----------------------------------------------------------------------
Yes, you can return your order within 30 days from the delivery date, which means you have until September 14, 2023, to initiate the return. Please ensure that the item is in its original packaging. Once the return is processed, the refund will take 5-7 business days to be credited back to your original payment method.


## Format 4: Plain text
Plain text is structured prose - full sentences, no special syntax, no markers. It is the most natural format for content that will be read or summarized, and the lowest friction to produce. It is what we get when we clean up raw retrieval output into clear, readable sentences without adding any markup.

**When it works well:**
- Conversational agents where the context is a short summary.
- Content that is already good prose (policy documents, knowledge base articles).
- When we want the model to reply in a natural, flowing style.

**When to avoid it:**
- When there are many distinct facts that need to be individually identifiable..
- When field-level precision matters (use JSON instead).
- When context mixes many different sources and types of information.

In [8]:
def format_as_plain_text(customer: dict, order: dict, policy: dict) -> str:
    """
    Format retrieved context as plain prose.
    Writes facts as clear, natural sentences without any markup.

    Args:
        customer: Customer profile data
        order: Order details
        policy: Return and warranty policy

    Returns:
        Plain text context string
    """
    return (
        f"The customer is {customer['name']}, a {customer['tier']}. "
        f"Their email is {customer['email']}. "
        f"They ordered {order['product']} (order {order['id']}) on {order['date']} for {order['price']}. "
        f"The order has been {order['status'].lower()} via {order['carrier']}. "
        f"The return policy allows returns within {policy['return_window']}, "
        f"provided the item is in {policy['condition'].lower()}. "
        f"Refunds are processed within {policy['refund_timeline']}. "
        f"The product comes with a {policy['warranty_duration']} warranty covering {policy['warranty_covers'].lower()}, "
        f"but not {policy['warranty_excludes'].lower()}."
    )


plain_context = format_as_plain_text(customer_data, order_data, policy_data)
prompt = f"{plain_context}\n\nQuestion: {TASK}"

response = llm.invoke([HumanMessage(content=prompt)])

print("PLAIN TEXT FORMATTED CONTEXT")
print("=" * 70)
print(plain_context)
print("\nMODEL RESPONSE:")
print("-" * 70)
print(response.content)

PLAIN TEXT FORMATTED CONTEXT
The customer is Sarah Chen, a Gold member since 2022. Their email is sarah@example.com. They ordered Wireless Headphones XZ-200 (order #12345) on 2023-08-15 for $299 USD. The order has been shipped via UPS (tracking: 1Z999AA10123456784). The return policy allows returns within 30 days from delivery date, provided the item is in original packaging required. Refunds are processed within 5-7 business days to original payment method. The product comes with a 2 years warranty covering manufacturing defects, but not physical damage.

MODEL RESPONSE:
----------------------------------------------------------------------
Yes, you can return your order. Since you ordered the Wireless Headphones XZ-200 on August 15, 2023, you have until September 14, 2023, to initiate a return, as the return policy allows returns within 30 days from the delivery date. Make sure the item is in its original packaging.

Once the return is processed, refunds typically take 5-7 business d

Plain text is the leanest markup-free option. It uses few tokens and produces the most conversational response. When our agent is chatting with a user and does not need to extract specific fields, this format is often the right choice.

## Comparing the five formats
Now let's put all five formats side-by-side to make the trade-offs concrete. We measure token count (a proxy for cost and latency), structural clarity and human readability.

In [10]:
def approximate_token_count(text: str) -> int:
    """
    Rough token count estimate: ~4 characters per token.
    Good enough for comparison purposes.

    Args:
        text: The text to estimate token count for

    Returns:
        Estimated token count
    """
    return len(text) // 4


formats = {
    "Raw (unformatted)": raw_context,
    "Markdown": markdown_context,
    "JSON": json_context,
    "XML": xml_context,
    "TOON": toon_context,
    "Plain text": plain_context,
}

print("FORMAT COMPARISON")
print("=" * 70)
print(f"{'Format':<22} {'Approx tokens':>15} {'Characters':>12}")
print("-" * 52)

for name, text in formats.items():
    tokens = approximate_token_count(text)
    chars = len(text.strip())
    print(f"{name:<22} {tokens:>15} {chars:>12}")

FORMAT COMPARISON
Format                   Approx tokens   Characters
----------------------------------------------------
Raw (unformatted)                  110          441
Markdown                           141          564
JSON                               170          682
XML                                194          777
TOON                                99          399
Plain text                         133          532


## Decision guide: Which format to use?
Use this decision guide when choosing a format for our retrieved context:

```
START: What is the nature of my context?
│
├─ Is it already structured data from a tool, API, or database?
│  └─ YES → Use JSON
│
├─ Does it mix structured fields with long prose sections?
│  └─ YES → Use XML
│
├─ Is token cost the primary constraint (large volume, tight budget)?
│  └─ YES → Use TOON
│
├─ Is it a short summary (< 5 facts) for a conversational agent?
│  └─ YES → Use plain text
│
├─ Does it have multiple logical sections of similar-length content?
│  └─ YES → Use Markdown
│
└─ Not sure? → Default to Markdown
   It is readable, widely understood by models, and works for most cases.
```

**Practical summary:**

| Format | Best for | Avoid when |
|---|---|---|
| **Markdown** | Multiple sections, mixed content | You need programmatic output parsing |
| **JSON** | API/tool outputs, field extraction | Content is narrative or conversational |
| **XML** | Mixed prose + structured data | Token budget is tight |
| **TOON** | Token-constrained pipelines, large volumes | Humans need to read or debug the context |
| **Plain text** | Short context, conversational agents | Many distinct facts need to be separate |

The most important thing is to pick one format and apply it consistently. A context block that mixes Markdown headers with raw JSON with XML tags is harder for the model than any single format used uniformly.

In [11]:
# Quick reference: a single function that applies the right format based on content type
def format_context(data: dict, content_type: str) -> str:
    """
    Apply the appropriate format based on the type of content.

    Args:
        data: Dictionary of data to format
        content_type: One of 'api_output', 'mixed', 'token_constrained',
                      'conversational', 'multi_section'

    Returns:
        Formatted context string
    """
    if content_type == "api_output":
        # Structured data from tools or APIs → JSON
        return json.dumps(data, indent=2)

    elif content_type == "mixed":
        # Mix of structured fields and prose → XML
        lines = ["<context>"]
        for key, value in data.items():
            lines.append(f"  <{key}>{value}</{key}>")
        lines.append("</context>")
        return "\n".join(lines)

    elif content_type == "token_constrained":
        # Token budget is tight → TOON
        # Each top-level key becomes a [section], its fields become key=value|key=value
        lines = []
        for section, content in data.items():
            tag = section.lower()[:6]  # Short section label
            if isinstance(content, dict):
                fields = "|".join(
                    f"{k[:5]}={v}" for k, v in content.items()
                )
                lines.append(f"[{tag}] {fields}")
            else:
                lines.append(f"[{tag}] {content}")
        return "\n".join(lines)

    elif content_type == "conversational":
        # Short summary for a chat agent → Plain text
        return " ".join(f"{v}." for v in data.values())

    elif content_type == "multi_section":
        # Multiple distinct sections → Markdown
        lines = []
        for section, content in data.items():
            lines.append(f"## {section}")
            if isinstance(content, dict):
                for k, v in content.items():
                    lines.append(f"- **{k}:** {v}")
            else:
                lines.append(str(content))
            lines.append("")
        return "\n".join(lines)

    else:
        raise ValueError(
            f"Unknown content_type: '{content_type}'. "
            "Use 'api_output', 'mixed', 'token_constrained', 'conversational', or 'multi_section'."
        )


# Example: same data, all five content types
sample = {
    "Customer": {"name": "Sarah Chen", "tier": "Gold"},
    "Order": {"id": "#12345", "status": "Shipped"},
    "Policy": {"returns": "30 days", "refund": "5-7 business days"}
}

for ctype in ["api_output", "mixed", "token_constrained", "multi_section"]:
    print(f"Content type: {ctype}")
    print("-" * 50)
    print(format_context(sample, ctype))
    print()

Content type: api_output
--------------------------------------------------
{
  "Customer": {
    "name": "Sarah Chen",
    "tier": "Gold"
  },
  "Order": {
    "id": "#12345",
    "status": "Shipped"
  },
  "Policy": {
    "returns": "30 days",
    "refund": "5-7 business days"
  }
}

Content type: mixed
--------------------------------------------------
<context>
  <Customer>{'name': 'Sarah Chen', 'tier': 'Gold'}</Customer>
  <Order>{'id': '#12345', 'status': 'Shipped'}</Order>
  <Policy>{'returns': '30 days', 'refund': '5-7 business days'}</Policy>
</context>

Content type: token_constrained
--------------------------------------------------
[custom] name=Sarah Chen|tier=Gold
[order] id=#12345|statu=Shipped
[policy] retur=30 days|refun=5-7 business days

Content type: multi_section
--------------------------------------------------
## Customer
- **name:** Sarah Chen
- **tier:** Gold

## Order
- **id:** #12345
- **status:** Shipped

## Policy
- **returns:** 30 days
- **refund:** 5-7 